# Deep Dive: Resource Estimation for Q# Code

The Q# code in this task implements Deutsch-Jozsa algorithm for two example functions (one constant and one balanced) - the same ones as in the debugging task. This task focuses on resource estimation for quantum algorithms, using a very simple example.

> [QDK documentation on resource estimation](https://docs.microsoft.com/en-us/quantum/user-guide/machines/resources-estimator).

1. Run the next cell using `Ctrl+Enter` (`⌘+Enter` on macOS) to compile the code.

In [ ]:
// Note that Q# code in the notebook shouldn't be defined inside a namespace.
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;

// Phase oracle implementing a constant function f(x) = 0
operation PhaseOracleZero (inputRegister : Qubit[]) : Unit {
    // Do nothing!
}

// Marking oracle implementing a balanced function f(x) = xₖ (the value of k-th bit)
operation MarkingOracleKthBit (inputRegister : Qubit[], target : Qubit, k : Int) : Unit {
    Controlled X([inputRegister[k]], target);
}

operation ApplyMarkingOracleAsPhaseOracle (
    markingOracle : ((Qubit[], Qubit) => Unit), 
    inputRegister : Qubit[]
) : Unit {
    using (target = Qubit()) {
        // Put the target into the |-⟩ state
        X(target);
        H(target);
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        markingOracle(inputRegister, target);
        // Return the target back to the |0⟩ state before releasing
        H(target);
        X(target);
    }
}

operation IsFunctionConstant (nQubits : Int, phaseOracle : (Qubit[] => Unit)) : Bool {
    mutable isConstant = true;
    using (qubits = Qubit[nQubits]) {
        // Apply the H gates, the oracle and the H gates again
        within {
            ApplyToEachA(H, qubits);
        } apply {
            phaseOracle(qubits);
        }
        // Measure all qubits
        let measurementResults = MultiM(qubits);
        for (m in measurementResults) {
            if (m == One) {
                set isConstant = false;
            }
        }
    }
    return isConstant;
}

function ConstantOrBalanced (value : Bool) : String {
    return value ? "constant" | "balanced";
}

// Note that Q# code in the notebook doesn't need @EntryPoint().
// Instead, we'll call the operation with %simulate in the next cell.
operation RunDeutschJozsaAlgorithm () : Unit {
    // for balanced function
    let phaseOracleKthBit = ApplyMarkingOracleAsPhaseOracle(MarkingOracleKthBit(_, _, 1), _);
    let isKthBitConstant = IsFunctionConstant(2, phaseOracleKthBit);
    Message($"f(x) = xk classified as {ConstantOrBalanced(isKthBitConstant)}");
}

2. `%simulate` runs the algorithm on a full state quantum simulator

In [ ]:
%simulate RunDeutschJozsaAlgorithm

3. `%trace` plots the circuit diagram of one algorithm run. (If the algorithm has branches depending on input parameters or measurement results, it will plot only one scenario.)

In [ ]:
%trace RunDeutschJozsaAlgorithm

4. `%estimate` runs resource estimation on the algorithm (without simulating the algorithm itself) and outputs the statistics.

In [ ]:
%estimate RunDeutschJozsaAlgorithm